In [52]:
import xarray as xr
import numpy as np

In [53]:
fp = "/beegfs/CMIP6/jdpaul3/cmip6_indicators_for_rasdaman/cmip6_indicators_ensemble.nc"
ds = xr.open_dataset(fp, decode_cf=True, mask_and_scale=False)

In [54]:
ds

<xarray.Dataset> Size: 6GB
Dimensions:      (model: 14, scenario: 5, time: 151, lat: 43, lon: 288)
Coordinates:
  * lat          (lat) float64 344B 90.0 89.06 88.12 87.17 ... 52.3 51.36 50.42
  * lon          (lon) float64 2kB -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
  * scenario     (scenario) int64 40B 0 1 2 3 4
  * model        (model) int64 112B 0 1 2 3 4 5 6 7 8 9 10 11 12 13
  * time         (time) datetime64[ns] 1kB 1950-01-01 1951-01-01 ... 2100-01-01
Data variables: (12/13)
    cd           (model, scenario, time, lat, lon) float32 524MB ...
    cdd          (model, scenario, time, lat, lon) int32 524MB ...
    csdi         (model, scenario, time, lat, lon) int32 524MB ...
    cwd          (model, scenario, time, lat, lon) int32 524MB ...
    dw           (model, scenario, time, lat, lon) int32 524MB ...
    ftc          (model, scenario, time, lat, lon) int32 524MB ...
    ...           ...
    r10mm        (model, scenario, time, lat, lon) int32 524MB ...
    rx1day       (model, scenario, time, lat, lon) int32 524MB ...
    rx5day       (model, scenario, time, lat, lon) int32 524MB ...
    su           (model, scenario, time, lat, lon) int32 524MB ...
    wsdi         (model, scenario, time, lat, lon) int32 524MB ...
    spatial_ref  int64 8B ...
Attributes:
    Conventions:  CF-1.8
    title:        CMIP6 Climate Indicator Data on a Common Grid with Multi-Mo...
    description:  Climate Indicator data from 13 CMIP6 models on a common gri...
    institution:  Scenarios Network for Alaska and Arctic Planning, Universit...
    source:       CMIP6 model output
    contact:      uaf-snap-data-tools@alaska.edu
    url:          https://uaf-snap.org/

In [55]:
# test each indicator to make sure the ensemble mean is calculated correctly

# historical data

lat = 65
lon = -142
time = "2014-01-01T00:00:00.000000000"
scenario = 0
models = slice(0, 13)  # first 13 models


for var in list(ds.data_vars):
    if var != "spatial_ref":
        print(f"Indicator: {var}")

        print("Original Dataset Values:")
        values = (
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=models)
            .values
        )

        print(values)
        print("\n Calculated Ensemble Mean:")
        # skip -9999 when calculating the mean
        values = np.where(values == -9999, np.nan, values)
        print(np.nanmean(values))

        print("Dataset Ensemble Mean:")
        print(
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=13)
            .values
        )

        print("\n\n")

Indicator: cd
Original Dataset Values:
[  nan -46.  -37.3 -40.4 -40.1 -44.8 -45.9 -39.8 -31.9 -40.3 -42.8 -37.
 -43.7 -40.8]

 Calculated Ensemble Mean:
-40.83077
Dataset Ensemble Mean:
-40.8



Indicator: cdd
Original Dataset Values:
[15 23 16 30 19 17 15 40 28 23 18 13 23 22]

 Calculated Ensemble Mean:
21.571428571428573
Dataset Ensemble Mean:
22



Indicator: csdi
Original Dataset Values:
[-9999     0     0     0     0     8     0     0     0     0     7     0
     0     1]

 Calculated Ensemble Mean:
1.2307692307692308
Dataset Ensemble Mean:
1



Indicator: cwd
Original Dataset Values:
[ 9  7  7 13 15  9 14 10 15 10 10 13  6 11]

 Calculated Ensemble Mean:
10.642857142857142
Dataset Ensemble Mean:
11



Indicator: dw
Original Dataset Values:
[-9999    62    18    36    28    56    82    44    13    21    39    22
    28    37]

 Calculated Ensemble Mean:
37.38461538461539
Dataset Ensemble Mean:
37



Indicator: ftc
Original Dataset Values:
[-9999    77    89    73    84    37    5

In [56]:
# test each indicator to make sure the ensemble mean is calculated correctly

# scenario data

lat = 65
lon = -142
time = "2020-01-01T00:00:00.000000000"
scenario = 1
models = slice(0, 13)  # first 13 models


for var in list(ds.data_vars):
    if var != "spatial_ref":
        print(f"Indicator: {var}")

        print("Original Dataset Values:")
        values = (
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=models)
            .values
        )

        print(values)
        print("\n Calculated Ensemble Mean:")
        # skip -9999 when calculating the mean
        values = np.where(values == -9999, np.nan, values)
        print(np.nanmean(values))

        print("Dataset Ensemble Mean:")
        print(
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=13)
            .values
        )

        print("\n\n")

Indicator: cd
Original Dataset Values:
[-39.4 -44.6   nan -41.5 -36.5 -44.9 -48.  -36.5 -27.4 -38.4 -39.2 -34.2
 -47.5 -39.8]

 Calculated Ensemble Mean:
-39.838463
Dataset Ensemble Mean:
-39.8



Indicator: cdd
Original Dataset Values:
[   25    35 -9999    23    14    30    26    26    20    26    11    21
    22    23]

 Calculated Ensemble Mean:
23.23076923076923
Dataset Ensemble Mean:
23



Indicator: csdi
Original Dataset Values:
[-9999     0 -9999    14     0     0     8     0     0     0     6     0
     6     3]

 Calculated Ensemble Mean:
3.0833333333333335
Dataset Ensemble Mean:
3



Indicator: cwd
Original Dataset Values:
[   13     7 -9999     7    17    13     6     7    11    10    16    13
     6    10]

 Calculated Ensemble Mean:
10.461538461538462
Dataset Ensemble Mean:
10



Indicator: dw
Original Dataset Values:
[   18    77 -9999    34    22    59    56    18     3    43    14    13
    63    35]

 Calculated Ensemble Mean:
35.0
Dataset Ensemble Mean:
35



Indicat